In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.worldfootball.net/schedule/eng-premier-league-2024-2025/"

def scrape_worldfootball(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    tables = soup.find_all("table")

    if not tables:
        raise Exception("No tables found on the page")

    matches = []

    for table in tables:
        rows = table.find_all("tr")

        for row in rows:
            cols = row.find_all("td")
            if len(cols) < 6:
                continue

            date = cols[0].get_text(strip=True)
            home = cols[2].get_text(strip=True)
            away = cols[4].get_text(strip=True)
            score = cols[5].get_text(strip=True)

            # Only finished matches have scores like "2:1"
            if ":" not in score:
                continue

            try:
                home_goals, away_goals = score.split(":")
            except ValueError:
                continue

            matches.append({
                "date": date,
                "home_team": home,
                "away_team": away,
                "home_goals": int(home_goals),
                "away_goals": int(away_goals)
            })

    return pd.DataFrame(matches)


df = scrape_worldfootball(URL)

print("Rows scraped:", len(df))
print(df.head())



HTTPError: 403 Client Error: Forbidden for url: https://www.worldfootball.net/schedule/eng-premier-league-2024-2025/